In [ ]:
!nvidia-smi

Sun Jan 17 11:07:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !pip install tez
# !pip install efficientnet_pytorch
# !pip uninstall albumentations
# !pip install albumentations

# Imports

In [ ]:
import os
import albumentations
import  matplotlib.pyplot as plt
import pandas as pd

import tez

from tez.datasets import ImageDataset
from tez.callbacks import EarlyStopping

import torch
import torch.nn as nn

import torchvision

from sklearn import metrics, model_selection
from efficientnet_pytorch import EfficientNet
from pathlib import Path
import argparse
import os

import albumentations
import pandas as pd
import tez
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
from sklearn import metrics, model_selection, preprocessing
from tez.callbacks import EarlyStopping, Callback
from tez.datasets import ImageDataset
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

import numpy as np

# Dataset Class

In [ ]:
# =====================================================================
# Dataset                                                        =
# =====================================================================


class ImageVecDataset(Dataset):

    def __init__(self, IMAGE_PATH, DF_PATH):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
        """
        self.image_dir = IMAGE_PATH
        self.df = pd.read_csv(DF_PATH)
        self.images = [str(i) + '.npy' for i in self.df.id.tolist()]
        self.labels = None
        if 'gender' in self.df.columns.tolist():
            self.genderLabels = self.df.gender.tolist()
            self.masterCategoryLabels = self.df.masterCategory.tolist()
            self.subCategoryLabels = self.df.subCategory.tolist()
            self.articleTypeLabels = self.df.articleType.tolist()
            self.baseColourLabels = self.df.baseColour.tolist()
            self.seasonLabels = self.df.season.tolist()
            self.usageLabels = self.df.usage.tolist()
            self.labels = True
        else:
            self.labels = None

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
      filename =self.images[idx]
      
      
      img = np.load(os.path.join(self.image_dir, filename))
      img = torch.from_numpy(img)
      if self.labels is not None:
          
          genderLabel = torch.tensor(self.genderLabels[idx])
          masterCategoryLabel = torch.tensor(self.masterCategoryLabels[idx])
          subCategoryLabel = torch.tensor(self.subCategoryLabels[idx])
          articleTypeLabel = torch.tensor(self.articleTypeLabels[idx])
          baseColourLabel = torch.tensor(self.baseColourLabels[idx])
          seasonLabel = torch.tensor(self.seasonLabels[idx])
          usageLabel = torch.tensor(self.usageLabels[idx])

        #   return {'image': img, 'genderLabel': genderLabel, 'seasonLabel': seasonLabel}

          return {'image': img, 'genderLabel': genderLabel, 'masterCategoryLabel': masterCategoryLabel, 'subCategoryLabel': subCategoryLabel, 
                        'articleTypeLabel': articleTypeLabel, 'baseColourLabel': baseColourLabel, 'seasonLabel': seasonLabel, 'usageLabel': usageLabel
          }
      return {'image': img}

# Model

In [ ]:
# =====================================================================
# Model                                                        =
# =====================================================================


class FashionModel(tez.Model):
    def __init__(self, num_classes):
        super().__init__()

        # self.effnet = EfficientNet.from_pretrained("efficientnet-b3")
        in_features = 1536
        # Layer 1
        self.bn1 = nn.BatchNorm1d(num_features=in_features)
        self.dropout1 = nn.Dropout(0.25)
        self.linear1 = nn.Linear(in_features=in_features, out_features=512, bias=False)
        self.relu = nn.ReLU()
        # Layer 2
        self.bn2 = nn.BatchNorm1d(num_features=512)
        self.dropout2 = nn.Dropout(0.5)

        self.gender = nn.Linear(512, num_classes['gender'])
        self.masterCategory = nn.Linear(512, num_classes['masterCategory'])
        self.subCategory = nn.Linear(512, num_classes['subCategory'])
        self.articleType = nn.Linear(512, num_classes['articleType'])
        self.baseColour = nn.Linear(512, num_classes['baseColour'])
        self.season = nn.Linear(512, num_classes['season'])
        self.usage = nn.Linear(512, num_classes['usage'])
       
        self.step_scheduler_after = "epoch"


    def monitor_metrics(self, outputs, targets):
        if targets is None:
            return {}
        accuracy = []
        for k,v in outputs.items():
            out = outputs[k]
            targ = targets[k]
            # print(out)
            out = torch.argmax(out, dim=1).cpu().detach().numpy()
            targ = targ.cpu().detach().numpy()
            accuracy.append(metrics.accuracy_score(targ, out))
        return {'accuracy': sum(accuracy)/len(accuracy)}

    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=3e-4)
        return opt

    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        return sch

    def forward(self, image, genderLabel=None, masterCategoryLabel=None, subCategoryLabel=None, articleTypeLabel=None, baseColourLabel=None, seasonLabel=None, usageLabel=None):
        batch_size, _ = image.shape

        x = self.linear1(self.dropout1(self.bn1(image)))

        x = self.relu(x)

        x = self.dropout2(self.bn2(x))

        targets = {}
        if genderLabel is None:
            targets = None
        else:
            targets['gender'] = genderLabel
            targets['masterCategory'] = masterCategoryLabel
            targets['subCategory'] = subCategoryLabel
            targets['articleType'] = articleTypeLabel
            targets['baseColour'] = baseColourLabel
            targets['season'] = seasonLabel
            targets['usage'] = usageLabel
        outputs = {}
        outputs["gender"] = self.gender(x)
        outputs["masterCategory"] = self.masterCategory(x)
        outputs["subCategory"] = self.subCategory(x)
        outputs["articleType"] = self.articleType(x)
        outputs["baseColour"] = self.baseColour(x)
        outputs["season"] = self.season(x)
        outputs["usage"] = self.usage(x)
        loss = []
        for k,v in outputs.items():
            loss.append(nn.CrossEntropyLoss()(outputs[k], targets[k]))
        loss = sum(loss)
        metrics = self.monitor_metrics(outputs, targets)
        return outputs, loss, metrics
    
    def extract_features(self, image):
        batch_size, _ = image.shape

        features = self.linear1(self.dropout1(self.bn1(image)))

        return features

In [ ]:

class SaveEfterEpoch(Callback):
    def __init__(self, model_path, model_name='epoch', save_interval=5):
        self.save_interval = save_interval
        self.model_path = model_path
        self.model_name = model_name
    
    def on_epoch_end(self, model):
        if model.current_epoch % self.save_interval == 0:
            model.save(os.path.join(self.model_path, f"{self.model_name}_{model.current_epoch}.bin"))
            print(f"model saved at epoch: {model.current_epoch}")

# Trainer

In [ ]:
BASE_DIR = Path('/content/drive/MyDrive/upwork/img')
IMAGE_VECTOR_PATH = Path('/content/drive/MyDrive/upwork/img/data/fashion-dataset/image_vectors')
IMAGE_PATH = Path('/content/drive/MyDrive/upwork/img/data/fashion-dataset/images')
MODEL_PATH = BASE_DIR/'models'

MODEL_NAME = 'fd_multi_output_classifier_v1'
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 30


image_path_list = list(Path(IMAGE_PATH).glob('*.jpg'))
print('Number of images: ', len(image_path_list))
dfx = pd.read_csv(BASE_DIR/'df_final.csv')

train_d = ImageVecDataset(IMAGE_PATH=IMAGE_VECTOR_PATH, DF_PATH=BASE_DIR/'train.csv')
train_dl = DataLoader(train_d, batch_size=TRAIN_BATCH_SIZE, shuffle=False, num_workers=8)

val_d = ImageVecDataset(IMAGE_PATH=IMAGE_VECTOR_PATH, DF_PATH=BASE_DIR/'test.csv')
val_dl = DataLoader(val_d, batch_size=VALID_BATCH_SIZE, shuffle=False, num_workers=8)


class_dict = dict([[i, dfx[i].nunique()] for i in dfx.columns.tolist()[1:]])

model = FashionModel(num_classes=class_dict)

es = EarlyStopping(
    monitor="valid_loss",
    model_path=os.path.join(MODEL_PATH, MODEL_NAME + ".bin"),
    patience=5,
    mode="min",
)

sfe = SaveEfterEpoch(save_interval=5, model_path=MODEL_PATH, model_name='fashion_model_epoch')

model.fit(
    train_d,
    valid_dataset=val_d,
    train_bs=TRAIN_BATCH_SIZE,
    valid_bs=VALID_BATCH_SIZE,
    device='cuda',
    epochs=EPOCHS,
    callbacks=[es, sfe],
    fp16=False,
)

model.save(os.path.join(MODEL_PATH, MODEL_NAME + "_last.bin"))

Number of images:  29163


  0%|          | 0/724 [00:00<?, ?it/s]

Validation score improved (inf --> 4.074792371929021). Saving model!
model saved at epoch: 0


  0%|          | 0/724 [00:00<?, ?it/s]

Validation score improved (4.074792371929021 --> 3.5499677052155385). Saving model!


...

EarlyStopping counter: 3 out of 5


  0%|          | 0/724 [00:00<?, ?it/s]

EarlyStopping counter: 4 out of 5


100%|██████████| 181/181 [00:06<00:00, 26.47it/s, accuracy=0.859, loss=2.97, stage=valid]


EarlyStopping counter: 5 out of 5


# Fature Extractor

In [ ]:
from tqdm import tqdm

BASE_DIR = Path('/content/drive/MyDrive/upwork/img')
IMAGE_VECTOR_PATH = Path('/content/drive/MyDrive/upwork/img/data/fashion-dataset/image_vectors')
IMAGE_PATH = Path('/content/drive/MyDrive/upwork/img/data/fashion-dataset/images')
MODEL_PATH = BASE_DIR/'models'

MODEL_NAME = 'fd_multi_output_classifier_v1'
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 30


image_path_list = list(Path(IMAGE_PATH).glob('*.jpg'))
print('Number of images: ', len(image_path_list))
dfx = pd.read_csv(BASE_DIR/'df_final.csv')


class_dict = dict([[i, dfx[i].nunique()] for i in dfx.columns.tolist()[1:]])

model = FashionModel(num_classes=class_dict)
model.eval()
model.cuda()

Number of images:  29163


FashionModel(
  (bn1): BatchNorm1d(1536, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.25, inplace=False)
  (linear1): Linear(in_features=1536, out_features=512, bias=False)
  (relu): ReLU()
  (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (gender): Linear(in_features=512, out_features=5, bias=True)
  (masterCategory): Linear(in_features=512, out_features=5, bias=True)
  (subCategory): Linear(in_features=512, out_features=28, bias=True)
  (articleType): Linear(in_features=512, out_features=75, bias=True)
  (baseColour): Linear(in_features=512, out_features=44, bias=True)
  (season): Linear(in_features=512, out_features=4, bias=True)
  (usage): Linear(in_features=512, out_features=7, bias=True)
)

In [ ]:
# =====================================================================
# Dataset                                                        =
# =====================================================================


class ImageVecExtractDataset(Dataset):

    def __init__(self, IMAGE_PATH, DF_PATH):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
        """
        self.image_dir = IMAGE_PATH
        self.df = pd.read_csv(DF_PATH)
        self.images = [str(i) + '.npy' for i in self.df.id.tolist()]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
      filename =self.images[idx]
      
      
      img = np.load(os.path.join(self.image_dir, filename))
      img = torch.from_numpy(img)
      
      return {'image': img, 'filename': filename}

In [ ]:
image_path_list = list(Path(IMAGE_VECTOR_PATH).glob('*.npy'))
print('Number of images: ', len(image_path_list))
OUTPUT_PATH = '/content/drive/MyDrive/upwork/img/data/fashion-dataset/image_vector_512'

f_dataset = ImageVecExtractDataset(IMAGE_PATH=IMAGE_VECTOR_PATH, DF_PATH=BASE_DIR/'df_final.csv')
f_dl = DataLoader(f_dataset, batch_size=32, shuffle=False, num_workers=8)

model.load(os.path.join(os.path.join(MODEL_PATH, MODEL_NAME + ".bin")))

for i, data in tqdm(enumerate(f_dl), total=len(f_dl)):
  # print(data)
  image_vector = model.extract_features(data['image'].cuda()).cpu().detach().numpy()
  for i in range(len(data['filename'])):
    f_name = data['filename'][i]
    np.save(os.path.join(OUTPUT_PATH, f_name), image_vector[i])

Number of images:  29162


100%|██████████| 905/905 [02:24<00:00,  6.25it/s]


In [ ]:
# image_path_list = list(Path(OUTPUT_PATH).glob('*.npy'))
# print('Number of images: ', len(image_path_list))


In [ ]:
# np.load(image_path_list[0]).shape

# Pipeline

In [3]:
import os

In [4]:
# !rm -rf '/content/drive/MyDrive/upwork/img/data/fashion-dataset/image_vector_512'
# !rm -rf '/content/drive/MyDrive/upwork/img/data/fashion-dataset/image_vectors'

In [5]:
root = '/content/drive/MyDrive/upwork/img'

In [6]:
import os, sys
print( os.getcwd())
os.chdir(root)
print( os.getcwd())

/content
/content/drive/.shortcut-targets-by-id/120/img


In [7]:
!ls

data					 model_transfer.py
df_final.csv				 multi_output_classifier.ipynb
eda.ipynb				 styles.csv
fashion_dataset_feature_extractor.ipynb  test.csv
kaggle.json				 test_dataset
model1_new				 train.csv
models					 transfer.ipynb


In [ ]:
# ! pwd

# ### UNbuffer
# ! nohup  python3 model_transfer.py  &

In [ ]:
# !python '/content/drive/MyDrive/upwork/img/model_transfer.py'

In [9]:
!python3 model_transfer.py

['1', '2', '3']
Starting Stage 1 -------------------------
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth
100% 47.1M/47.1M [00:00<00:00, 63.6MB/s]
Loaded pretrained weights for efficientnet-b3
100% 3619/3619 [1:12:02<00:00,  1.19s/it]
Stage 1 Finished -------------------------
Starting Stage 2 -------------------------
100% 2895/2895 [07:29<00:00,  6.44it/s, accuracy=0.728, loss=6.26, stage=train]
100% 724/724 [01:54<00:00,  6.34it/s, accuracy=0.814, loss=3.98, stage=valid]
Validation score improved (inf --> 3.981799150369444). Saving model!
model saved at epoch: 0
100% 2895/2895 [00:55<00:00, 52.30it/s, accuracy=0.78, loss=4.6, stage=train]
100% 724/724 [00:09<00:00, 74.08it/s, accuracy=0.823, loss=3.71, stage=valid]
Validation score improved (3.981799150369444 --> 3.7102390160547434). Saving model!
100% 2895/2895 [00:55<00:00, 51.99it/s, accuracy=0.7

In [10]:
import torch 
print(torch.__version__)

1.7.0+cu101


In [13]:
!python3 extract_vectors_big_model.py

Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408
100% 1810/1810 [43:48<00:00,  1.45s/it]
